# Confection de résumés automatiques d'articles scientifiques par un modèle de langue pré-entraîné : étude et comparaison des performances de deux modèles.

**Auteur.e.s :** Sophie perrin, Emma, El Zube, Marius, Sylvano, pour le cours *Representation learning for NLP* @ Master 2 MALIA et MIASHS.

**Equipe :** les léopards

## Préparation de l'environnement

1. Se connecter ou se créer un compte sur https://huggingface.co/
2. Se créer un nouveau token d'accès : https://huggingface.co/settings/tokens
3. Enregistrer ce token comme un secret nommé `HF_TOKEN` dans Google Colab (icone "clef" dans le bandeau vertical)
4. Exécuter le code ci-dessous

Note importante pour la vitesse de calcul : pour activer l'accélération GPU dans votre notebook Kaggle :

    Ouvrez le menu « Settings » : Dans votre notebook Kaggle, cliquez sur le menu « Accelerator » en haut de l'écran, et choisissez un GPU.

    Dans ce même menu, vérifiez qu'il n'est pas écrit "turn off internet" - sinon cliquer dessus pour rétablir l'accès au reste d'internet depuis kaggle. Si vous n'avez ni "turn off internet" ni "turn on internet" d'écrit, alors il faut ajouter votre téléphone et votre photo pour certifier votre identité pour le compte, avant de pouvoir accéder au reste d'internet depuis kaggle (nécessaire pour charger les modèles de huggingface et même les packages de python...!).



In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

m_2_maliash_resume_darticles_scientifiques_path = kagglehub.competition_download('m-2-maliash-resume-darticles-scientifiques')

print('Data source import complete.')

Data source import complete.


In [ ]:
#Pour créer le "HF_TOKEN" dans Kaggle, aller dans "Add-ons" (dans les menus de ce notebook, juste à gauche de "Help").
#Une fois là, aller dans "secrets", puis ça marche à peu près comme dans colab : il faut nommer la variable (HF_TOKEN) et
#y copier sa valeur.


from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")


# Préparation des données d'entraînement pour leur utilisation par le modèle

## Données du fichier OBS

On crée un dictionnaire qui apparie les articles de l'ensemble d'entraînement ("fine tuning" puisque le modèle est déjà pré-entraîné) et leurs résumés par leur identifiant commun.

A noter que quelques résumés n'ont pas d'article qui leur correspond !

In [ ]:
import os

# Chemins dans Kaggle des dossiers contenant les fichiers
dossier_abstracts = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/train/OBS/abstracts_OBS/"
dossier_articles = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/train/OBS/articles_OBS/"


# Liste des fichiers dans chaque dossier
fichiers_abstracts = [f for f in os.listdir(dossier_abstracts) if f.startswith("abstract-")]
fichiers_articles = [f for f in os.listdir(dossier_articles) if f.startswith("article-")]

# Dictionnaires pour stocker les fichiers par identifiant
abstracts = {}
articles = {}

# Remplir les dictionnaires avec les fichiers en fonction des identifiants
for fichier in fichiers_abstracts:
    # Extraire l'identifiant du fichier abstract
    identifiant = fichier.split("-")[1].split(".")[0]
     #fichier.split("-") : La méthode split("-") divise le nom du fichier
    #en une liste de sous-chaînes en utilisant le caractère "-" comme séparateur.
    #Par exemple, si le fichier est "abstract-123.txt", fichier.split("-") renverra la liste ["abstract", "123.txt"].
    #fichier.split("-")[1] : En prenant l'élément d'indice 1 de cette liste (c'est-à-dire "123.txt"), on obtient la partie du nom du fichier après le préfixe "abstract-".
    #split(".")[0] : Ensuite, on divise cette chaîne "123.txt" avec split("."), ce qui donne ["123", "txt"].
    #On prend le premier élément de la liste (c'est-à-dire "123") qui est l'identifiant unique de l'abstract.
    abstracts[identifiant] = fichier
    #Cette ligne ajoute une entrée dans le dictionnaire abstracts, où la clé est : identifiant (par exemple "123")
    #et la valeur est le nom du fichier : fichier (par exemple "abstract-123.txt").

for fichier in fichiers_articles:
    # Extraire l'identifiant du fichier article
    identifiant = fichier.split("-")[1].split(".")[0]
    articles[identifiant] = fichier

# Dictionnaire pour stocker les appariements
appariements = {}

# Liste des abstracts et articles non-appariés
non_apparies_abstracts = []
non_apparies_articles = []

# Appariement des fichiers abstracts et articles par identifiant
for identifiant in abstracts:
    if identifiant in articles:
        # Ajouter l'appariement au dictionnaire
        appariements[identifiant] = {
            "abstract": abstracts[identifiant],
            "article": articles[identifiant]
        }
    else:
        # Ajouter à la liste des abstracts non appariés
        non_apparies_abstracts.append(abstracts[identifiant])

# Vérifier les articles non-appariés
for identifiant in articles:
    if identifiant not in abstracts:
        # Ajouter à la liste des articles non appariés
        non_apparies_articles.append(articles[identifiant])

# Affichage des appariements
"""
print("Appariements :")
for identifiant, fichiers in appariements.items():
    print(f"Identifiant {identifiant}:")
    print(f"  Abstract: {fichiers['abstract']}")
    print(f"  Article: {fichiers['article']}")
"""
# Affichage des abstracts non-appariés
print("\nAbstracts non-appariés :")
for abstract in non_apparies_abstracts:
    print(abstract)

# Affichage des articles non-appariés
print("\nArticles non-appariés :")
for article in non_apparies_articles:
    print(article)


Abstracts non-appariés :
abstract-36404343.txt
abstract-38022520.txt
abstract-37614109.txt
abstract-36944082.txt
abstract-32017677.txt
abstract-36944050.txt
abstract-36891751.txt
abstract-35081022.txt
abstract-32091358.txt
abstract-31625835.txt
abstract-36066965.txt
abstract-36519326.txt
abstract-36525381.txt
abstract-36314570.txt
abstract-37833688.txt
abstract-27423055.txt
abstract-37796016.txt
abstract-37026745.txt
abstract-36068298.txt
abstract-35324507.txt
abstract-37726286.txt
abstract-37102598.txt
abstract-36951168.txt
abstract-24279685.txt
abstract-36944043.txt
abstract-32946618.txt
abstract-36372692.txt

Articles non-appariés :


On retravaille un peu la forme de ce dictionnaire pour pouvoir le convertir au format "dataset" de Hugging face, nécessaire pour pouvoir entraîner le modèle choisi dessus.


In [ ]:
!pip install -U datasets #le "-U" demande l'installation de la toute dernière version du package
from datasets import Dataset #importe la classe Dataset de la bibliothèque datasets de Hugging Face.
#Cette classe est utilisée pour manipuler des ensembles de données dans un format qui peut être utilisé pour l'entraînement de modèles issus d'Hugging Face.

# Limiter le nombre de threads de JAX à 1 (pour éviter les problèmes dans kaggle)
os.environ["JAX_NUM_THREADS"] = "1"

#Notre dictionnaire appariements n'est pas tout à fait sous la bonne forme pour être transformé en dataset de Hugging Face :
#on le remanie donc pour qu'il ait cette bonne forme.

# Initialisation des listes vides
identifiants = []
articles = []
abstracts = []

# Remplir les listes avec les données extraites du dictionnaire appariements
for identifiant, values in appariements.items():
    identifiants.append(identifiant)         # Ajout de l'identifiant
    articles.append(values["article"])      # Ajout de l'article
    abstracts.append(values["abstract"])    # Ajout de l'abstract

# Créer un dictionnaire avec les listes
data = {
    "identifiant": identifiants,
    "article": articles,
    "abstract": abstracts
}

# Créer le Dataset en convertissant notre nouveau dictionnaire via l'instruction Dataset.from_dict() de Dataset
dataset = Dataset.from_dict(data)

print(dataset)

Dataset({
    features: ['identifiant', 'article', 'abstract'],
    num_rows: 402
})


## le fichier à utiliser pour l'entraînement du modèle sur les articles de type OBS sera donc "dataset"

# Données RCT

On procède de même pour les données des articles de type "RCT"

In [ ]:
import os

# Chemins dans Kaggle des dossiers contenant les fichiers
dossier_abstracts = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/train/RCT/abstracts_RCT/"
dossier_articles = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/train/RCT/articles_RCT/"


# Liste des fichiers dans chaque dossier
fichiers_abstracts = [f for f in os.listdir(dossier_abstracts) if f.startswith("abstract-")]
fichiers_articles = [f for f in os.listdir(dossier_articles) if f.startswith("article-")]

# Dictionnaires pour stocker les fichiers par identifiant
abstracts = {}
articles = {}

# Remplir les dictionnaires avec les fichiers en fonction des identifiants
for fichier in fichiers_abstracts:
    # Extraire l'identifiant du fichier abstract
    identifiant = fichier.split("-")[1].split(".")[0]
     #fichier.split("-") : La méthode split("-") divise le nom du fichier
    #en une liste de sous-chaînes en utilisant le caractère "-" comme séparateur.
    #Par exemple, si le fichier est "abstract-123.txt", fichier.split("-") renverra la liste ["abstract", "123.txt"].
    #fichier.split("-")[1] : En prenant l'élément d'indice 1 de cette liste (c'est-à-dire "123.txt"), on obtient la partie du nom du fichier après le préfixe "abstract-".
    #split(".")[0] : Ensuite, on divise cette chaîne "123.txt" avec split("."), ce qui donne ["123", "txt"].
    #On prend le premier élément de la liste (c'est-à-dire "123") qui est l'identifiant unique de l'abstract.
    abstracts[identifiant] = fichier
    #Cette ligne ajoute une entrée dans le dictionnaire abstracts, où la clé est : identifiant (par exemple "123")
    #et la valeur est le nom du fichier : fichier (par exemple "abstract-123.txt").

for fichier in fichiers_articles:
    # Extraire l'identifiant du fichier article
    identifiant = fichier.split("-")[1].split(".")[0]
    articles[identifiant] = fichier

# Dictionnaire pour stocker les appariements
appariements_RCT = {}

# Liste des abstracts et articles non-appariés
non_apparies_abstracts_RCT = []
non_apparies_articles_RCT = []

# Appariement des fichiers abstracts et articles par identifiant
for identifiant in abstracts:
    if identifiant in articles:
        # Ajouter l'appariement au dictionnaire
        appariements_RCT[identifiant] = {
            "abstract": abstracts[identifiant],
            "article": articles[identifiant]
        }
    else:
        # Ajouter à la liste des abstracts non appariés
        non_apparies_abstracts_RCT.append(abstracts[identifiant])

# Vérifier les articles non-appariés
for identifiant in articles:
    if identifiant not in abstracts:
        # Ajouter à la liste des articles non appariés
        non_apparies_articles_RCT.append(articles[identifiant])

# Affichage des appariements
"""
print("Appariements_RCT :")
for identifiant, fichiers in appariements_RCT.items():
    print(f"Identifiant {identifiant}:")
    print(f"  Abstract: {fichiers['abstract']}")
    print(f"  Article: {fichiers['article']}")
"""
# Affichage des abstracts non-appariés
print("\nAbstracts non-appariés :")
for abstract in non_apparies_abstracts_RCT:
    print(abstract)

# Affichage des articles non-appariés
print("\nArticles non-appariés :")
for article in non_apparies_articles_RCT:
    print(article)


Abstracts non-appariés :
abstract-24164420.txt
abstract-36082590.txt
abstract-36872899.txt
abstract-37030393.txt
abstract-37722926.txt
abstract-36451616.txt
abstract-36094045.txt
abstract-37157134.txt
abstract-36877135.txt
abstract-36289532.txt
abstract-24266855.txt
abstract-37565064.txt
abstract-37614106.txt
abstract-33153517.txt
abstract-37562034.txt
abstract-37690911.txt
abstract-31431084.txt
abstract-33984185.txt
abstract-24268098.txt
abstract-36542086.txt
abstract-36053287.txt
abstract-32418064.txt
abstract-37302021.txt
abstract-37463508.txt
abstract-31507265.txt
abstract-33674243.txt
abstract-36271420.txt
abstract-36129998.txt
abstract-36525381.txt
abstract-29172800.txt
abstract-37392348.txt
abstract-32868525.txt
abstract-28961557.txt
abstract-37551774.txt
abstract-35986699.txt
abstract-27421672.txt
abstract-34713539.txt
abstract-36503413.txt
abstract-35751625.txt
abstract-35982483.txt
abstract-36309392.txt
abstract-35974668.txt
abstract-30394151.txt
abstract-37217019.txt
abstra

In [ ]:
!pip install -U datasets #le "-U" demande l'installation de la toute dernière version du package
from datasets import Dataset #importe la classe Dataset de la bibliothèque datasets de Hugging Face.
#Cette classe est utilisée pour manipuler des ensembles de données dans un format qui peut être utilisé pour l'entraînement de modèles issus d'Hugging Face.

# Limiter le nombre de threads de JAX à 1 (pour éviter les problèmes dans kaggle)
os.environ["JAX_NUM_THREADS"] = "1"

#Notre dictionnaire appariements_RCT n'est pas tout à fait sous la bonne forme pour être transformé en dataset de Hugging Face :
#on le remanie donc pour qu'il ait cette bonne forme.

# Initialisation des listes vides
identifiants = []
articles = []
abstracts = []

# Remplir les listes avec les données extraites du dictionnaire appariements
for identifiant, values in appariements_RCT.items():
    identifiants.append(identifiant)         # Ajout de l'identifiant
    articles.append(values["article"])      # Ajout de l'article
    abstracts.append(values["abstract"])    # Ajout de l'abstract

# Créer un dictionnaire avec les listes
data = {
    "identifiant": identifiants,
    "article": articles,
    "abstract": abstracts
}

# Créer le Dataset en convertissant notre nouveau dictionnaire via l'instruction Dataset.from_dict() de Dataset
dataset_RCT = Dataset.from_dict(data)

#print(dataset_RCT)

# Le fichier à utiliser pour l'entrainement sur les données RCT sera donc "dataset_RCT"

# Essai du modèle bigbird-pegasus-large-arxiv

Ce modèle a pour singularité, par rapport à des modèles de type BERT, d'utiliser une version modifiée de l'architecture Transformer : il a un mécanisme d'attention sparse (attention réduite) par blocs. Concrètement, cela signifie qu'au lieu de calculer l'attention entre toutes les positions du texte, il divise la séquence (le texte) en blocs et applique l'attention uniquement au sein de ces blocs, ainsi qu'entre certains blocs (par exemple les blocs voisins ou un échantillonnage aléatoire de blocs).

Cela permet d'être économe en temps de calcul : avec un modèle de type BERT, ce dernier croissait en $O(N^2)$, où $N$ représente la taille du texte (ou document). Avec le mécanisme d'attention sparse par blocs, on tombe à des temps de calcul en $O(N.log(N))$ ou en $O(N)$.

Pour nos articles scientifiques, qui sont des documents longs, ce type de modèles, avec attention sparse par blocs, peut donc être particulièrement pertinent.

On convertit maintenant le texte des articles et des résumés (ici ceux de type "OBS") en séquences de tokens pour le modèle bigbird-pegasus-large-arxiv.

In [ ]:
"""
# Fonction pour préparer les données (tokenisation) pour un modèle chargé depuis huggingface
def tokenize_function(examples):
    # Tokeniser les articles et les résumés
    inputs = tokenizer(examples['article'], truncation=True, padding="max_length", max_length=1024) #données en entrée
    #examples['article'] : C'est la donnée brute (l'article) fournie à la fonction sous forme de texte.
    #Cette donnée vient d'un Dataset ou d'une DataLoader et est passée sous forme de liste de textes (articles).
    #tokenizer(examples['article']) : Cette ligne utilise le tokenizer pour convertir le texte brut des articles en une séquence de tokens.
    #Le tokenizer transforme le texte en un format compréhensible par le modèle (i.e., des IDs de tokens).

    #truncation=True : Si le texte est plus long que la longueur maximale définie, il sera tronqué pour correspondre à cette longueur maximale.

    #padding="max_length" : Cela permet de remplir (padd) le texte pour qu'il atteigne la longueur maximale spécifiée.
    #Si un article est plus court que la longueur maximale, des tokens de remplissage seront ajoutés.

    #max_length=1024 : La longueur maximale des séquences d'entrée est fixée à 1024 tokens. Si un article est plus long que cela, il sera tronqué à 1024 tokens.

    targets = tokenizer(examples['abstract'], truncation=True, padding="max_length", max_length=256)#données cible de l'entrainement du modèle

    # Retourner les inputs et targets sous la forme de dictionnaires
    inputs["labels"] = targets["input_ids"]
    #inputs["labels"] : Dans un modèle de génération de texte comme BigBirdPegasus, les labels sont les séquences que le modèle doit prédire
    #(les résumés dans ce cas). En d'autres termes, le modèle apprend à prédire les tokens du résumé à partir des tokens de l'article.
    #targets["input_ids"] : input_ids est la représentation des tokens du résumé (les IDs numériques des tokens). Ces tokens sont utilisés comme labels lors de l'entraînement,
    #c'est-à-dire que le modèle essaiera de prédire ces input_ids lorsqu'il verra les tokens des articles.
    #Cette ligne ajoute donc les input_ids des résumés dans la clé "labels" des données d'entrée, qui est utilisée pendant l'entraînement pour calculer la perte
    #entre les prédictions du modèle et les résumés réels.
    return inputs
"""

'\n# Fonction pour préparer les données (tokenisation) pour un modèle chargé depuis huggingface\ndef tokenize_function(examples):\n    # Tokeniser les articles et les résumés\n    inputs = tokenizer(examples[\'article\'], truncation=True, padding="max_length", max_length=1024) #données en entrée\n    #examples[\'article\'] : C\'est la donnée brute (l\'article) fournie à la fonction sous forme de texte.\n    #Cette donnée vient d\'un Dataset ou d\'une DataLoader et est passée sous forme de liste de textes (articles).\n    #tokenizer(examples[\'article\']) : Cette ligne utilise le tokenizer pour convertir le texte brut des articles en une séquence de tokens.\n    #Le tokenizer transforme le texte en un format compréhensible par le modèle (i.e., des IDs de tokens).\n\n    #truncation=True : Si le texte est plus long que la longueur maximale définie, il sera tronqué pour correspondre à cette longueur maximale.\n\n    #padding="max_length" : Cela permet de remplir (padd) le texte pour qu\'i

## Chargement du modèle bigbird-pegasus-large-arxiv



In [ ]:
"""
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
import torch
from transformers import TrainingArguments, Trainer # Import TrainingArguments and Trainer

#Instancier le tokenizer pour le modèle pegasus
#tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/bigbird-pegasus-large-arxiv/")
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")
#chargement du modèle :
# by default encoder-attention is `block_sparse` with num_random_blocks=3, block_size=64/par défaut l'encoder-attention est "block sparse"
#avec num_random_blocks=3 et block_size=64
#model = BigBirdPegasusForConditionalGeneration.from_pretrained("/kaggle/input/bigbird-pegasus-large-arxiv/",
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv",
                                                               return_dict=True,
                                                               torch_dtype=torch.float16,
                                                               device_map="auto")
"""
######################Options pour modifier la taille et le nombre des blocs d'attention :
# decoder attention type can't be changed & will be "original_full"
# you can change `attention_type` (encoder only) to full attention like this:
##model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", attention_type="original_full")

# you can change `block_size` & `num_random_blocks` like this:
##model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", block_size=16, num_random_blocks=2)
######################



'\nfrom transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer\nimport torch\nfrom transformers import TrainingArguments, Trainer # Import TrainingArguments and Trainer\n\n#Instancier le tokenizer pour le modèle pegasus\n#tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/bigbird-pegasus-large-arxiv/")\ntokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")\n#chargement du modèle :\n# by default encoder-attention is `block_sparse` with num_random_blocks=3, block_size=64/par défaut l\'encoder-attention est "block sparse"\n#avec num_random_blocks=3 et block_size=64\n#model = BigBirdPegasusForConditionalGeneration.from_pretrained("/kaggle/input/bigbird-pegasus-large-arxiv/",\nmodel = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv",\n                                                               return_dict=True,\n                                                               torch_dtype=torch.flo

## Entrainement du modèle bigbird-pegasus-large-arxiv et conclusions sur ce modèle pour notre compétition kaggle

On paramètre l'entraînement du modèle.

Problème : même en modifiant les paramètres de training_args comme ci-dessous, pour tenter de consommer moins de mémoire, l'entraînement de ce modèle fait planter la session kaggle car il consomme trop de mémoire - y compris en utilisant les GPU 100, ceux qui comportent le plus de mémoire sur kaggle.

Nous allons donc tester un autre modèle, composé par l'assemblage de bart-large-cnn et longformer-base-4096, et bart-large-cnn seul (simplement fine-tuné sur nos données d'entraînement).

In [ ]:
"""
# Appliquer la tokenisation
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./bigbird_pegasus_finetuned",  # Répertoire de sortie pour enregistrer les résultats
    evaluation_strategy="epoch",  # Évaluer après chaque époque
    learning_rate=2e-5,  # Taux d'apprentissage
    per_device_train_batch_size=1, #2 je réduis pour éviter de planter kaggle,  # Taille du batch pour l'entraînement
    per_device_eval_batch_size=1, #2 idem,   # Taille du batch pour l'évaluation
    gradient_accumulation_steps=8,  # Accumuler les gradients sur 8 batches avant la mise à jour (réduit l'utilisation de la mémoire, toujours pour éviter de planter kaggle)
                                    #cela "simule" une taille de batch plus grande que ce qu'elle n'est en réalité ici
    fp16=False,  # Activer l'entraînement en 16-bit - toujours pour éviter de planter kaggle
    num_train_epochs=3,  # Nombre d'époques d'entraînement
    dataloader_num_workers=4,  # Utiliser 4 processus pour charger les données en parallèle
    gradient_checkpointing=True,  # Activer gradient checkpointing
    save_steps=500,  # Sauvegarder moins souvent
    weight_decay=0.01,  # L2 regularization
    save_total_limit=2,  # Limite du nombre de sauvegardes du modèle
    logging_dir="./logs",  # Répertoire pour les logs
    logging_steps=100,  # Fréquence des logs
    report_to="tensorboard",  # Optionnel : Utiliser TensorBoard pour la visualisation des métriques
)

# Créer un objet Trainer
trainer = Trainer(
    model=model,  # Le modèle à fine-tuner
    args=training_args,  # Les arguments d'entraînement
    train_dataset=train_dataset,  # Jeu d'entraînement
    eval_dataset=val_dataset,  # Jeu de validation
    compute_metrics=compute_metrics,  # Fonction pour calculer les métriques - elle nous permettra d'utiliser le score "ROUGE"
)

# Lancer l'entraînement
trainer.train()


# Sauvegarder le modèle fine-tuné
trainer.save_model("./bigbird_pegasus_finetuned")

"""

'\n# Appliquer la tokenisation\ntokenized_datasets = dataset.map(tokenize_function, batched=True)\n\n# Définir les arguments d\'entraînement\ntraining_args = TrainingArguments(\n    output_dir="./bigbird_pegasus_finetuned",  # Répertoire de sortie pour enregistrer les résultats\n    evaluation_strategy="epoch",  # Évaluer après chaque époque\n    learning_rate=2e-5,  # Taux d\'apprentissage\n    per_device_train_batch_size=1, #2 je réduis pour éviter de planter kaggle,  # Taille du batch pour l\'entraînement\n    per_device_eval_batch_size=1, #2 idem,   # Taille du batch pour l\'évaluation\n    gradient_accumulation_steps=8,  # Accumuler les gradients sur 8 batches avant la mise à jour (réduit l\'utilisation de la mémoire, toujours pour éviter de planter kaggle)\n                                    #cela "simule" une taille de batch plus grande que ce qu\'elle n\'est en réalité ici\n    fp16=False,  # Activer l\'entraînement en 16-bit - toujours pour éviter de planter kaggle\n    num_t

# Essai du modèle Bart-large-cnn seul

BART est un modèle de transformer encodeur-décodeur (seq2seq : "sequence to séquence") avec un encodeur bidirectionnel (similaire à BERT) et un décodeur autoregressif (similaire à GPT).
Il est pré-entraîné en masquant des parties du texte et en apprenant à prédire ces parties - comme si on lui demandait de débruiter un texte bruité.
Il utilise une attention dense classique pour un transformer, où chaque token de l'entrée prend en compte tous les autres tokens de la séquence. Cela le handicape donc pour nos articles longs (comme tous les modèles à attention dense classique).

Compléter ici avec les conclusions qui nous ont fait ne pas retenir ce modèle seul non plus...

# Confection d'un modèle composé à partir de longformer et bart

## Chargement du modèle  allenai/longformer-base-4096



Le modèle bigbird-pegasus-large-arivx demande trop de mémoire pour notre plateforme de compétition kaggle.

Nous devons donc nous orienter vers un modèle de compromis, néanmoins adapté au traitement d'articles scientifiques longs.
Il nous faut donc chercher dans la famille des modèles à attention "sparse", les seuls vraiment adaptés pour ces textes longs.

allenai/longformer-large-4096 partage avec bigbird ce mécanisme d'attention sparse : tout comme lui, il utilise une fenêtre d'attention "glissante". Tout comme lui, il y ajoute une attention à plus longue portée sur les tokens importants, ce qui lui permet d'interagir avec tous les autres tokens de la séquence pour capturer des relations à long terme. Mais en revanche, contrairement à lui, il n'a pas de mécanisme d'attention aléatoire qui se surajoute à ces deux couches d'attention, et qui lui permet de mieux capter des relations globales...mais consomme de la mémoire vive, lors de son entraînement (fine tuning) notamment.

En revanche, il n'est pas un générateur de texte : il nous faut donc l'associer à un modèle de type GPT ou autre qui permet quant à lui la génération. Ce, dans le but de bonifier les performances du modèle générateur par les bonnes performances de longformer sur les textes longs.
On l'associe avec le modèle générateur bart-large-cnn, association qu'on crée "à la main" via la confection d'une classe spécifique.

Comme le modèle ainsi créé reste trop volumineux, on y remplace finalement allenai/longformer-large-4096 par une version plus légère : allenai/longformer-base-4096.



In [ ]:
from transformers import LongformerForSequenceClassification, AutoTokenizer
import torch
from transformers import TrainingArguments, Trainer # Import TrainingArguments and Trainer

# Instancier le tokenizer pour le modèle Longformer
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

# Chargement du modèle Longformer
# Longformer n'a pas de modèle direct pour la génération de texte comme BigBird-PEGASUS, donc ici, nous utilisons un modèle Longformer pour classification de séquences.
# Si vous voulez adapter Longformer pour la génération de texte, vous devrez peut-être fine-tuner un modèle approprié sur une tâche comme la génération.

model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096",
                                                           return_dict=True,
                                                           torch_dtype=torch.float16,
                                                           device_map="auto")

# Si vous avez besoin de fine-tuner ce modèle sur une tâche spécifique (comme le résumé), vous devrez remplacer le modèle par un modèle adapté à la génération.

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Création du nouveau modèle "cousu main" à partir de BART et LONGFORMER

On est obligé de faire un peu de "cuisine" pour associer les 2 modèles en un seul : par exemple, les sorties de longformer n'ont pas la même taille que les entrées de bart. Il nous faut donc insérer dans la classe une "couche de projection".
Il faut également dupliquer leurs poids, pour éviter les (mauvais...) mélanges entre eux par la suite.

On ne peut pas utiliser la tokenisation de l'un des deux modèles non plus : il nous faut en créer une, tout aussi hybride que notre modèle "fait main" à partir de la couture de ces deux modèles issus de hugging face.

In [ ]:
from transformers import LongformerTokenizer, BartTokenizer, BartForConditionalGeneration, LongformerModel, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.cuda.empty_cache()

# Charger le tokenizer et le modèle Longformer pour l'encodage
longformer_tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
longformer_model = LongformerModel.from_pretrained('allenai/longformer-base-4096')

# Charger le tokenizer et le modèle BART-CNN pour la génération
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Tokenisation des entrées et des sorties
def tokenize_function(examples):
    # Tokeniser les articles avec Longformer
    model_inputs = longformer_tokenizer(
        examples["article"],
        #max_length=4096, #trop long pour le GPU de kaggle...
        max_length=1024,
        truncation=True,
        padding="max_length",  # Assurer le padding
        return_tensors="pt"  # Retourner des tensors PyTorch
    )

    # Tokeniser les résumés (labels) avec BART tokenizer
    labels = bart_tokenizer(
        #examples["highlights"],
        examples["abstract"],
        #max_length=200,
        max_length=1024,
        truncation=True,
        padding="max_length",  # Assurer le padding
        return_tensors="pt"  # Retourner des tensors PyTorch
    )

    # Ajouter les labels au dictionnaire de sortie
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Définir un modèle combiné qui utilise Longformer comme encodeur et BART pour la génération
class LongformerBart(nn.Module):
    def __init__(self, longformer_model, bart_model):
        super(LongformerBart, self).__init__()
        self.longformer = longformer_model
        self.bart = bart_model

 # Ajouter une couche linéaire pour ajuster la dimension des sorties de Longformer
        self.longformer_projection = nn.Linear(768, 1024)  # Projeter de 768 à 1024

 # Dupliquer les poids partagés de BART et Longformer
        with torch.no_grad():
            # Dupliquer les poids partagés de BART
            bart_model.model.shared = torch.nn.Embedding.from_pretrained(bart_model.model.shared.weight.clone())
            bart_model.model.encoder.embed_tokens = torch.nn.Embedding.from_pretrained(bart_model.model.encoder.embed_tokens.weight.clone())
            bart_model.model.decoder.embed_tokens = torch.nn.Embedding.from_pretrained(bart_model.model.decoder.embed_tokens.weight.clone())

            # Dupliquer les poids partagés de Longformer
            longformer_model.embeddings.word_embeddings = torch.nn.Embedding.from_pretrained(longformer_model.embeddings.word_embeddings.weight.clone())

    def forward(self, input_ids, attention_mask, decoder_input_ids=None, labels=None):
        # Encoder la séquence avec Longformer
        encoder_outputs = self.longformer(input_ids=input_ids, attention_mask=attention_mask)
        encoder_hidden_states = encoder_outputs.last_hidden_state

  # Appliquer la projection pour ajuster la dimension
        encoder_hidden_states = self.longformer_projection(encoder_hidden_states)

        # Passer les représentations encodées à BART pour la génération
        decoder_outputs = self.bart(
            input_ids=decoder_input_ids,
            labels=labels,
            encoder_outputs=encoder_hidden_states
        )

        return decoder_outputs

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

# Entrainement de ce modèle sur les données OBS, puis sauvegarde de ce modèle ainsi fine-tuné


In [ ]:
# Appliquer la tokenisation
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Diviser l'ensemble de données en train et validation
dataset_split = tokenized_datasets.train_test_split(test_size=0.2)

# Extraire les ensembles d'entraînement et de validation
train_dataset = dataset_split['train']
val_dataset = dataset_split['test']



# Initialiser le modèle combiné
model = LongformerBart(longformer_model, bart_model)

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./longformer_bart_finetuned",  # Répertoire de sortie pour enregistrer les résultats
    evaluation_strategy="epoch",  # Évaluer après chaque époque
    learning_rate=2e-5,  # Taux d'apprentissage
    per_device_train_batch_size=2,  # Taille du batch pour l'entraînement
    per_device_eval_batch_size=2,   # Taille du batch pour l'évaluation
    num_train_epochs=3,  # Nombre d'époques d'entraînement
    save_steps=500,  # Sauvegarder moins souvent
    weight_decay=0.01,  # L2 regularization
    logging_dir="./logs",  # Répertoire pour les logs
    logging_steps=10,  # Fréquence des logs
    report_to="tensorboard",  # Optionnel : Utiliser TensorBoard pour la visualisation des métriques
)


# Créer un objet Trainer
trainer = Trainer(
    model=model,  # Le modèle combiné Longformer+BART
    args=training_args,  # Les arguments d'entraînement
    train_dataset=train_dataset,  # Jeu d'entraînement
    eval_dataset=val_dataset,  # Jeu de validation
    tokenizer=longformer_tokenizer,  # Tokenizer pour le pré-traitement des données
)

# Lancer l'entraînement
trainer.train()

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.061900,1.215846
2,0.044400,6.686377
3,0.042600,2.663265


TrainOutput(global_step=483, training_loss=0.5426618188434507, metrics={'train_runtime': 598.6016, 'train_samples_per_second': 1.609, 'train_steps_per_second': 0.807, 'total_flos': 0.0, 'train_loss': 0.5426618188434507, 'epoch': 3.0})

In [ ]:
# Dupliquer les poids partagés pour éviter la duplication de mémoire
# Cela doit être fait après l'entraînement et avant la sauvegarde

import safetensors.torch as st

# Pour Longformer - dupliquer les embeddings
with torch.no_grad():  # Empêche la modification des gradients
    new_word_embeddings = model.longformer.embeddings.word_embeddings.weight.clone()
    model.longformer.embeddings.word_embeddings = torch.nn.Embedding.from_pretrained(new_word_embeddings)

# Pour BART - dupliquer les embeddings de l'encodeur et du décodeur
with torch.no_grad():  # Empêche la modification des gradients
    new_shared_weight = model.bart.model.shared.weight.clone()
    model.bart.model.shared = torch.nn.Embedding.from_pretrained(new_shared_weight)

    new_encoder_embed = model.bart.model.encoder.embed_tokens.weight.clone()
    model.bart.model.encoder.embed_tokens = torch.nn.Embedding.from_pretrained(new_encoder_embed)

    new_decoder_embed = model.bart.model.decoder.embed_tokens.weight.clone()
    model.bart.model.decoder.embed_tokens = torch.nn.Embedding.from_pretrained(new_decoder_embed)

# Sauvegarder les poids dans un fichier classique
torch.save(model.state_dict(), 'longformer_bart_finetuned/model_weights.pth')

# Charger manuellement avec safetensors (si vous avez configuré safetensors)
import safetensors.torch as st

# Sauvegarder avec safetensors après avoir dupliqué les poids
st.save_file(model.state_dict(), 'longformer_bart_finetuned/model_weights.safetensors')

import json

# Définir la configuration
config = {
    "architectures": ["LongformerForSequenceClassification"],
    "hidden_size": 1024,
    "num_attention_heads": 16,
    "num_hidden_layers": 12,
    "vocab_size": 50264,
    "max_position_embeddings": 4098,
    "embedding_size": 1024,
    "layer_norm_eps": 1e-5,
    "pad_token_id": 1,
    "activation_function": "gelu",
    "initializer_range": 0.02,
    "layer_norm_affine": True,
    "attention_probs_dropout_prob": 0.1,
    "hidden_dropout_prob": 0.1,
    "max_position_embeddings": 4098,
    "type_vocab_size": 2,
    "attention_dropout": 0.1,
    "longformer_projection": {
        "in_features": 768,
        "out_features": 1024
    }
}

# Sauvegarder le fichier config.json
#config_path = '/kaggle/working/config.json'
with open("longformer_bart_finetuned/config.json", 'w') as f:
    json.dump(config, f)

print("Le fichier config.json a été sauvegardé ")

# Sauvegarder le modèle fine-tuné
trainer.save_model("./longformer_bart_finetuned")
"""
# Sauvegarde du modèle
torch.save(longformer_bart_model.state_dict(), "longformer_bart_finetuned.pth")

# Sauvegarde de la configuration
longformer_bart_model.config.to_json_file("longformer_bart_config.json")
"""

Le fichier config.json a été sauvegardé 


'\n# Sauvegarde du modèle\ntorch.save(longformer_bart_model.state_dict(), "longformer_bart_finetuned.pth")\n\n# Sauvegarde de la configuration\nlongformer_bart_model.config.to_json_file("longformer_bart_config.json")\n'

In [ ]:
import shutil
# Chemin du dossier à supprimer
directory_to_delete = "/kaggle/working/longformer_bart_finetuned/checkpoint-483"
# Vérifier si le dossier existe
if os.path.exists(directory_to_delete):
    # Supprimer le dossier et tout son contenu
    shutil.rmtree(directory_to_delete)
    print(f"Le dossier '{directory_to_delete}' a été supprimé avec succès.")
else:
    print(f"Le dossier '{directory_to_delete}' n'existe pas.")

Le dossier '/kaggle/working/longformer_bart_finetuned/checkpoint-483' a été supprimé avec succès.


On zipe le dossier du modèle ainsi créé et entraîné, pour pouvoir le sauvegarder et le réutiliser ensuite sur notre ensemble de test :


In [ ]:
"""
print(os.listdir("./"))

shutil.make_archive('/kaggle/working/longformer_bart_finetuned', 'zip', './longformer_bart_finetuned')
"""

['.virtual_documents', 'logs', 'longformer_bart_finetuned', 'submission.csv']


'/kaggle/working/longformer_bart_finetuned.zip'

# Entraînement de ce modèle sur les données RCT, puis sauvegarde de ce modèle fine-tuné

In [ ]:
# Appliquer la tokenisation
tokenized_datasets = dataset_RCT.map(tokenize_function, batched=True)

# Diviser l'ensemble de données en train et validation
dataset_split = tokenized_datasets.train_test_split(test_size=0.2)

# Extraire les ensembles d'entraînement et de validation
train_dataset = dataset_split['train']
val_dataset = dataset_split['test']



# Initialiser le modèle combiné
model = LongformerBart(longformer_model, bart_model)

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./longformer_bart_finetuned_rct",  # Répertoire de sortie pour enregistrer les résultats
    evaluation_strategy="epoch",  # Évaluer après chaque époque
    learning_rate=2e-5,  # Taux d'apprentissage
    per_device_train_batch_size=2,  # Taille du batch pour l'entraînement
    per_device_eval_batch_size=2,   # Taille du batch pour l'évaluation
    num_train_epochs=3,  # Nombre d'époques d'entraînement
    save_steps=500,  # Sauvegarder moins souvent
    weight_decay=0.01,  # L2 regularization
    logging_dir="./logs",  # Répertoire pour les logs
    logging_steps=10,  # Fréquence des logs
    report_to="tensorboard",  # Optionnel : Utiliser TensorBoard pour la visualisation des métriques
)


# Créer un objet Trainer
trainer = Trainer(
    model=model,  # Le modèle combiné Longformer+BART
    args=training_args,  # Les arguments d'entraînement
    train_dataset=train_dataset,  # Jeu d'entraînement
    eval_dataset=val_dataset,  # Jeu de validation
    tokenizer=longformer_tokenizer,  # Tokenizer pour le pré-traitement des données
)

# Lancer l'entraînement
trainer.train()

'\n# Appliquer la tokenisation\ntokenized_datasets = dataset_RCT.map(tokenize_function, batched=True)\n\n# Diviser l\'ensemble de données en train et validation\ndataset_split = tokenized_datasets.train_test_split(test_size=0.2)\n\n# Extraire les ensembles d\'entraînement et de validation\ntrain_dataset = dataset_split[\'train\']\nval_dataset = dataset_split[\'test\']\n\n        \n\n# Initialiser le modèle combiné\nmodel = LongformerBart(longformer_model, bart_model)\n\n# Définir les arguments d\'entraînement\ntraining_args = TrainingArguments(\n    output_dir="./longformer_bart_finetuned_rct",  # Répertoire de sortie pour enregistrer les résultats\n    evaluation_strategy="epoch",  # Évaluer après chaque époque\n    learning_rate=2e-5,  # Taux d\'apprentissage\n    per_device_train_batch_size=2,  # Taille du batch pour l\'entraînement\n    per_device_eval_batch_size=2,   # Taille du batch pour l\'évaluation\n    num_train_epochs=3,  # Nombre d\'époques d\'entraînement\n    save_steps=

In [ ]:
# Dupliquer les poids partagés pour éviter la duplication de mémoire
# Cela doit être fait après l'entraînement et avant la sauvegarde

import safetensors.torch as st

# Pour Longformer - dupliquer les embeddings
with torch.no_grad():  # Empêche la modification des gradients
    new_word_embeddings = model.longformer.embeddings.word_embeddings.weight.clone()
    model.longformer.embeddings.word_embeddings = torch.nn.Embedding.from_pretrained(new_word_embeddings)

# Pour BART - dupliquer les embeddings de l'encodeur et du décodeur
with torch.no_grad():  # Empêche la modification des gradients
    new_shared_weight = model.bart.model.shared.weight.clone()
    model.bart.model.shared = torch.nn.Embedding.from_pretrained(new_shared_weight)

    new_encoder_embed = model.bart.model.encoder.embed_tokens.weight.clone()
    model.bart.model.encoder.embed_tokens = torch.nn.Embedding.from_pretrained(new_encoder_embed)

    new_decoder_embed = model.bart.model.decoder.embed_tokens.weight.clone()
    model.bart.model.decoder.embed_tokens = torch.nn.Embedding.from_pretrained(new_decoder_embed)

# Sauvegarder les poids dans un fichier classique
torch.save(model.state_dict(), 'longformer_bart_finetuned_rct/model_weights.pth')

# Sauvegarder avec safetensors après avoir dupliqué les poids
st.save_file(model.state_dict(), 'longformer_bart_finetuned_rct/model_weights.safetensors')


from transformers import PretrainedConfig

import json

# Définir la configuration
config = {
    "architectures": ["LongformerForSequenceClassification"],
    "hidden_size": 1024,
    "num_attention_heads": 16,
    "num_hidden_layers": 12,
    "vocab_size": 50264,
    "max_position_embeddings": 4098,
    "embedding_size": 1024,
    "layer_norm_eps": 1e-5,
    "pad_token_id": 1,
    "activation_function": "gelu",
    "initializer_range": 0.02,
    "layer_norm_affine": True,
    "attention_probs_dropout_prob": 0.1,
    "hidden_dropout_prob": 0.1,
    "max_position_embeddings": 4098,
    "type_vocab_size": 2,
    "attention_dropout": 0.1,
    "longformer_projection": {
        "in_features": 768,
        "out_features": 1024
    }
}



# Sauvegarder le fichier config.json
with open("longformer_bart_finetuned_rct/config.json", 'w') as f:
    json.dump(config, f)

print("Le fichier config.json a été sauvegardé ")


# Sauvegarder le modèle fine-tuné
trainer.save_model("./longformer_bart_finetuned_rct")
"""
"""
# Sauvegarde du modèle
torch.save(longformer_bart_model.state_dict(), "longformer_bart_finetuned.pth")

# Sauvegarde de la configuration
longformer_bart_model.config.to_json_file("longformer_bart_config.json")

'\n# Sauvegarde du modèle\ntorch.save(longformer_bart_model.state_dict(), "longformer_bart_finetuned.pth")\n\n# Sauvegarde de la configuration\nlongformer_bart_model.config.to_json_file("longformer_bart_config.json")\n'

In [ ]:
import shutil
# Chemin du dossier à supprimer
directory_to_delete = "/kaggle/working/longformer_bart_finetuned_rct/checkpoint-393"
# Vérifier si le dossier existe
if os.path.exists(directory_to_delete):
    # Supprimer le dossier et tout son contenu
    shutil.rmtree(directory_to_delete)
    print(f"Le dossier '{directory_to_delete}' a été supprimé avec succès.")
else:
    print(f"Le dossier '{directory_to_delete}' n'existe pas.")

Le dossier '/kaggle/working/longformer_bart_finetuned_rct/checkpoint-393' n'existe pas.


On zipe le dossier du modèle ainsi créé et entraîné, pour pouvoir le sauvegarder et le réutiliser ensuite sur notre ensemble de test :

In [ ]:
"""
import shutil

print(os.listdir("./"))

shutil.make_archive('/kaggle/working/longformer_bart_finetuned_rct', 'zip', './longformer_bart_finetuned_rct')
"""

'\nimport shutil\n\nprint(os.listdir("./"))\n\nshutil.make_archive(\'/kaggle/working/longformer_bart_finetuned_rct\', \'zip\', \'./longformer_bart_finetuned_rct\')\n'

## Chargement de notre modèle fine-tuné sur les données "OBS" et constitué à partir des deux modèles bart et longformer, pour pouvoir l'utiliser pour l'ensemble de test et soumettre nos résumés pour la compétition

In [ ]:
#Charger le modèle fine tuné afin de l'utiliser
import torch
from transformers import LongformerModel, BartForConditionalGeneration, LongformerConfig
import torch.nn as nn


# Définir la classe LongformerBart
class LongformerBart(nn.Module):
    def __init__(self, longformer_model, bart_model):
        super(LongformerBart, self).__init__()
        self.longformer = longformer_model
        self.bart = bart_model
        self.longformer_projection = nn.Linear(768, 1024)  # Projeter de 768 à 1024

    def forward(self, input_ids, attention_mask, decoder_input_ids=None, labels=None):
        encoder_outputs = self.longformer(input_ids=input_ids, attention_mask=attention_mask)
        encoder_hidden_states = encoder_outputs.last_hidden_state
        encoder_hidden_states = self.longformer_projection(encoder_hidden_states)  # Appliquer la projection
        decoder_outputs = self.bart(input_ids=decoder_input_ids, labels=labels, encoder_outputs=encoder_hidden_states)
        return decoder_outputs

# Charger Longformer et BART
longformer_model = LongformerModel.from_pretrained('allenai/longformer-base-4096')
bart = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Charger les poids de notre modèle fine-tuné
model = LongformerBart(longformer_model, bart)
#model.load_state_dict(torch.load('/kaggle/input/longformer_bart_finetuned/pytorch/default/1/longformer_bart_finetuned/longformer_bart_finetuned.pth'))  # Remplacez par le bon chemin
model.load_state_dict(torch.load('/kaggle/working/longformer_bart_finetuned/model_weights.pth'))  # Remplacez par le bon chemin et la ligne du dessus pour utiliser le modèle déjà réentrainé

# Charger la configuration
# Notez que vous devez avoir la configuration JSON pour le modèle personnalisé
#config = LongformerConfig.from_json_file('/kaggle/input/longformer_bart_finetuned/pytorch/default/1/longformer_bart_finetuned/longformer_bart_finetuned_config.json')
config = LongformerConfig.from_json_file('/kaggle/working/longformer_bart_finetuned/config.json') #idem : # Remplacez par le bon chemin et la ligne du dessus pour utiliser le modèle déjà réentrainé



<ipython-input-21-88ae010882a4>:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/working/longformer_bart_finetuned/model_weights.p

## Mise en forme des données "OBS" de test pour le test du modèle.

In [ ]:
import os
import pandas as pd

# Définir le chemin du dossier contenant les articles
dossier_articles_test = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/test/OBS_test/articles_OBS_test"

# Liste des fichiers dans le dossier
fichiers_articles = [f for f in os.listdir(dossier_articles_test) if f.startswith("article-")]

# Dictionnaire pour stocker les articles par identifiant (avec contenu des fichiers)
articles_test = {}

# Remplir le dictionnaire avec les fichiers en fonction des identifiants
for fichier in fichiers_articles:
    # Extraire l'identifiant du fichier article
    identifiant = fichier.split("-")[1].split(".")[0]

    # Lire le contenu du fichier
    with open(os.path.join(dossier_articles_test, fichier), 'r') as f:
        contenu_article = f.read()

    # Ajouter l'article et son identifiant au dictionnaire
    articles_test[identifiant] = {"article": contenu_article}

# Convertir le dictionnaire en DataFrame
df_articles_test = pd.DataFrame.from_dict(articles_test, orient='index')

# Réinitialiser l'index pour que 'identifiant' devienne une colonne normale
df_articles_test.reset_index(inplace=True)

# Renommer les colonnes
df_articles_test.columns = ['identifiant', 'article']

# Vérifier le DataFrame
print(df_articles_test.head())


  identifiant                                            article
0    28278130  Cardiometabolic Risk Factors Among 1.3 Million...
1    34555924  Prostate Cancer Screening and Incidence among ...
2    35157313  The associated burden of mental health conditi...
3    36906849  Implementing digital systems to facilitate gen...
4    37226713  Patient-reported treatment response in chronic...


## Tokenisation des données et génération des résumés pour les articles de test de type "OBS"

In [ ]:
from transformers import LongformerTokenizer, BartTokenizer, BartForConditionalGeneration
import torch
import pandas as pd

# Initialiser les tokenizers Longformer et BART
longformer_tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Charger un modèle BART pour la génération de résumé
#model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
model = model

def tokenize_function(examples):
    """
    Tokenisation des articles et des résumés avec Longformer et BART respectivement.
    """
    # Si 'examples' est une chaîne de texte, placez-la sous une clé dans un dictionnaire
    if isinstance(examples, str):
        examples = {"article": examples}

    # Tokenisation de l'article avec Longformer
    model_inputs = longformer_tokenizer(
        examples["article"],
        max_length=1024,  # Longformer est capable de gérer de longs textes - mais kaggle ne supporte pas plus des 1024 tokens de long...
        truncation=True,
        padding="max_length",  # Assurer le padding
        return_tensors="pt"  # Retourner des tensors PyTorch
    )

    # Tokenisation des résumés avec BART
    labels = bart_tokenizer(
        examples["abstract"],  # Assume que "abstract" est la cible à tokenizer
        max_length=1024,
        truncation=True,
        padding="max_length",  # Assurer le padding
        return_tensors="pt"  # Retourner des tensors PyTorch
    )

    # Retourne les entrées et labels sous forme de dictionnaire
    return {**model_inputs, "labels": labels["input_ids"]}


def generer_resume(article, model, tokenizer, max_length=150):
    """
    Fonction pour générer un résumé d'un article en utilisant notre modèle Longformer-BART.
    """
    # Tokenisation de l'article
    inputs = tokenizer(article, return_tensors="pt", truncation=True, padding=True, max_length=1024)

    # Si le modèle est en mode CPU, déplacer les entrées et le modèle sur le bon périphérique
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Obtenir les sorties du modèle
    with torch.no_grad():
        # Appeler la méthode forward du modèle en utilisant les bons paramètres
        outputs = model(input_ids=inputs['input_ids'],
                        attention_mask=inputs['attention_mask'],  # Remarque: utilisez 'attention_mask' ici
                        decoder_input_ids=inputs['input_ids'],  # Utiliser les input_ids comme entrée pour le décodeur
                        labels=None)

    # Décoder le résumé depuis la sortie du modèle
    # Utilisation de la méthode de génération du modèle BART sur les sorties de Longformer
    summary_ids = model.bart.generate(
        inputs['input_ids'],
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )

    # Décoder les IDs générés pour obtenir le résumé
    resume = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return resume

def traiter_ensemble_test(dataset_RCT, model, tokenizer):
    """
    Fonction qui traite l'ensemble du dataset_RCT et génère des résumés pour chaque article.
    """
    # Convertir le dataset en DataFrame si ce n'est pas déjà fait
    df = pd.DataFrame(dataset_RCT)

    # Créer une nouvelle colonne pour les résumés générés
    df['résumé'] = df['article'].apply(lambda x: generer_resume(x, model, tokenizer))

    # Afficher le résultat ou sauvegarder les résultats dans un fichier
    return df[['identifiant', 'article', 'résumé']]


# Traiter l'ensemble de test et générer les résumés
resultats_obs = traiter_ensemble_test(df_articles_test, model, bart_tokenizer)

# Afficher les résultats
print(resultats_obs)

# Supprimer la colonne 'article' et renommer les colonnes dans 'resultats' (pour mise au bon format pour la compétition)
resultats_obs = resultats_obs.drop(columns=['article'])  # Supprimer la colonne 'article'
resultats_obs = resultats_obs.rename(columns={'identifiant': 'id', 'résumé': 'abstract'})  # Renommer les colonnes


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


   identifiant                                            article  \
0     28278130  Cardiometabolic Risk Factors Among 1.3 Million...   
1     34555924  Prostate Cancer Screening and Incidence among ...   
2     35157313  The associated burden of mental health conditi...   
3     36906849  Implementing digital systems to facilitate gen...   
4     37226713  Patient-reported treatment response in chronic...   
5     37450671  Prevalence of Immunomodulator Use as Combinati...   
6     37578507  Serum levels of endocannabinoids and related l...   
7     37580243  Impact of antimicrobial stewardship in organis...   
8     37586661  Effectiveness of intravenous immunoglobulin th...   
9     37698611  Virological evidence of the impact of non-phar...   
10    37870070  Clinical diagnosis of SARS-CoV-2 infection: An...   
11    37924841  Association of patient clinical and gut microb...   
12    38030736  Effects of maternal type 1 diabetes and confou...   
13    38180993  Serum lactate and 

## Chargement de notre modèle fine-tuné sur les données "RCT" et constitué à partir des deux modèles bart et longformer, pour pouvoir l'utiliser pour l'ensemble de test et soumettre nos résumés pour la compétition

In [ ]:
#Charger le modèle fine tuné afin de l'utiliser
import torch
from transformers import LongformerModel, BartForConditionalGeneration, LongformerConfig
import torch.nn as nn


# Définir la classe LongformerBart
class LongformerBart(nn.Module):
    def __init__(self, longformer_model, bart_model):
        super(LongformerBart, self).__init__()
        self.longformer = longformer_model
        self.bart = bart_model
        self.longformer_projection = nn.Linear(768, 1024)  # Projeter de 768 à 1024

    def forward(self, input_ids, attention_mask, decoder_input_ids=None, labels=None):
        encoder_outputs = self.longformer(input_ids=input_ids, attention_mask=attention_mask)
        encoder_hidden_states = encoder_outputs.last_hidden_state
        encoder_hidden_states = self.longformer_projection(encoder_hidden_states)  # Appliquer la projection
        decoder_outputs = self.bart(input_ids=decoder_input_ids, labels=labels, encoder_outputs=encoder_hidden_states)
        return decoder_outputs

# Charger Longformer et BART
longformer_model = LongformerModel.from_pretrained('allenai/longformer-base-4096')
bart = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Charger les poids de notre modèle fine-tuné
model = LongformerBart(longformer_model, bart)
model.load_state_dict(torch.load('/kaggle/input/longformer_bart_fine_tuned_rct/pytorch/default/1/model_weights.pth'))
#model.load_state_dict(torch.load('/kaggle/working/longformer_bart_finetuned_rct/model_weigths.pth')) # Remplacez par le bon chemin et cette ligne si vous réentrainez le modèle



# Charger la configuration
# Notez que vous devez avoir la configuration JSON pour le modèle personnalisé
config = LongformerConfig.from_json_file('/kaggle/input/longformer_bart_fine_tuned_rct/pytorch/default/1/config.json')
#config = LongformerConfig.from_json_file('/kaggle/working/longformer_bart_finetuned_rct/config.json') #idem : Remplacez par le bon chemin et cette ligne si vous réentrainez le modèle


<ipython-input-24-c5822b5fd9d3>:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/input/longformer_bart_fine_tuned_rct/pytorch/defa

In [ ]:
#Mise en forme des données à tester

import os
import pandas as pd

# Définir le chemin du dossier contenant les articles
dossier_articles_test = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/test/RCT_test/articles_RCT_test"

# Liste des fichiers dans le dossier
fichiers_articles = [f for f in os.listdir(dossier_articles_test) if f.startswith("article-")]

# Dictionnaire pour stocker les articles par identifiant (avec contenu des fichiers)
articles_test = {}

# Remplir le dictionnaire avec les fichiers en fonction des identifiants
for fichier in fichiers_articles:
    # Extraire l'identifiant du fichier article
    identifiant = fichier.split("-")[1].split(".")[0]

    # Lire le contenu du fichier
    with open(os.path.join(dossier_articles_test, fichier), 'r') as f:
        contenu_article = f.read()

    # Ajouter l'article et son identifiant au dictionnaire
    articles_test[identifiant] = {"article": contenu_article}

# Convertir le dictionnaire en DataFrame
df_articles_test = pd.DataFrame.from_dict(articles_test, orient='index')

# Réinitialiser l'index pour que 'identifiant' devienne une colonne normale
df_articles_test.reset_index(inplace=True)

# Renommer les colonnes
df_articles_test.columns = ['identifiant', 'article']
# Vérifier le DataFrame
print(df_articles_test.head())

  identifiant                                            article
0    26811968  Randomized Controlled Trial of Hospital-Based ...
1    34737472  Behavioural intervention for adolescent uptake...
2    34844893  Prediction of clinical response to corticoster...
3    34969647  Feasibility and efficacy of a multidisciplinar...
4    35532871  Iron homeostasis in heart transplant recipient...


# Tokenisation des données et génération des résumés pour les articles de test de type "RCT"

In [ ]:
from transformers import LongformerTokenizer, BartTokenizer, BartForConditionalGeneration
import torch
import pandas as pd

# Initialiser les tokenizers Longformer et BART
longformer_tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Charger un modèle BART pour la génération de résumé
#model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
model = model

def tokenize_function(examples):
    """
    Tokenisation des articles et des résumés avec Longformer et BART respectivement.
    """
    # Si 'examples' est une chaîne de texte, placez-la sous une clé dans un dictionnaire
    if isinstance(examples, str):
        examples = {"article": examples}

    # Tokenisation de l'article avec Longformer
    model_inputs = longformer_tokenizer(
        examples["article"],
        max_length=1024,  # Longformer est capable de gérer de longs textes - mais kaggle ne supporte pas plus des 1024 tokens de long...
        truncation=True,
        padding="max_length",  # Assurer le padding
        return_tensors="pt"  # Retourner des tensors PyTorch
    )

    # Tokenisation des résumés avec BART
    labels = bart_tokenizer(
        examples["abstract"],  # Assume que "abstract" est la cible à tokenizer
        max_length=1024,
        truncation=True,
        padding="max_length",  # Assurer le padding
        return_tensors="pt"  # Retourner des tensors PyTorch
    )

    # Retourne les entrées et labels sous forme de dictionnaire
    return {**model_inputs, "labels": labels["input_ids"]}


def generer_resume(article, model, tokenizer, max_length=150):
    """
    Fonction pour générer un résumé d'un article en utilisant notre modèle Longformer-BART.
    """
    # Tokenisation de l'article
    inputs = tokenizer(article, return_tensors="pt", truncation=True, padding=True, max_length=1024)

    # Si le modèle est en mode CPU, déplacer les entrées et le modèle sur le bon périphérique
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Obtenir les sorties du modèle
    with torch.no_grad():
        # Appeler la méthode forward du modèle en utilisant les bons paramètres
        outputs = model(input_ids=inputs['input_ids'],
                        attention_mask=inputs['attention_mask'],  # Remarque: utilisez 'attention_mask' ici
                        decoder_input_ids=inputs['input_ids'],  # Utiliser les input_ids comme entrée pour le décodeur
                        labels=None)

    # Décoder le résumé depuis la sortie du modèle
    # Utilisation de la méthode de génération du modèle BART sur les sorties de Longformer
    summary_ids = model.bart.generate(
        inputs['input_ids'],
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )

    # Décoder les IDs générés pour obtenir le résumé
    resume = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return resume

def traiter_ensemble_test(dataset_RCT, model, tokenizer):
    """
    Fonction qui traite l'ensemble du dataset_RCT et génère des résumés pour chaque article.
    """
    # Convertir le dataset en DataFrame si ce n'est pas déjà fait
    df = pd.DataFrame(dataset_RCT)

    # Créer une nouvelle colonne pour les résumés générés
    df['résumé'] = df['article'].apply(lambda x: generer_resume(x, model, tokenizer))

    # Afficher le résultat ou sauvegarder les résultats dans un fichier
    return df[['identifiant', 'article', 'résumé']]


# Traiter l'ensemble de test et générer les résumés
resultats_rct = traiter_ensemble_test(df_articles_test, model, bart_tokenizer)

# Afficher les résultats
print(resultats_rct)

# Supprimer la colonne 'article' et renommer les colonnes dans 'resultats' (pour mise au bon format pour la compétition)
resultats_rct = resultats_rct.drop(columns=['article'])  # Supprimer la colonne 'article'
resultats_rct = resultats_rct.rename(columns={'identifiant': 'id', 'résumé': 'abstract'})  # Renommer les colonnes

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


   identifiant                                            article  \
0     26811968  Randomized Controlled Trial of Hospital-Based ...   
1     34737472  Behavioural intervention for adolescent uptake...   
2     34844893  Prediction of clinical response to corticoster...   
3     34969647  Feasibility and efficacy of a multidisciplinar...   
4     35532871  Iron homeostasis in heart transplant recipient...   
5     35638317  The effect of duloxetine on mechanistic pain p...   
6     36479935  Clinical Predictors of Adherence to Exercise T...   
7     36621009  Supporting return to work after psychiatric ho...   
8     37972955  LIGHTSITE III: 13-Month Efficacy and Safety Ev...   
9     37991188  Evolution in Laparoscopic Gastrectomy From a R...   
10    38052803  Long-term outcome of children with acute promy...   
11    38060092  Efficacy and Safety of the Travoprost Intraocu...   
12    38117526  Efficacy and Safety of PF-07038124 in Patients...   
13    38184526  Prognostic signifi

# Fusion de nos deux fichiers de résumés générés (resultats_obs et resultats_rct) en un seul fichier submission, puis conversion en csv pour soumission à la compétition (permet d'avoir le score rouge).

In [ ]:
# Concaténer les deux DataFrames
resultats = pd.concat([resultats_obs, resultats_rct], ignore_index=True)

# Sauvegarder le DataFrame concaténé dans un fichier CSV, si nécessaire
resultats.to_csv('submission.csv', index=False)

# Afficher le DataFrame concaténé
print(resultats)

          id                                           abstract
0   28278130  Cardiometabolic Risk Factors Among 1.3 Million...
1   34555924  Prostate Cancer Screening and Incidence among ...
2   35157313  The associated burden of mental health conditi...
3   36906849  Implementing digital systems to facilitate gen...
4   37226713  Chronic graft-versus-host disease (GvHD) is th...
5   37450671  Prevalence of Immunomodulator Use as Combinati...
6   37578507  Serum levels of endocannabinoids and related l...
7   37580243  Impact of antimicrobial stewardship in organis...
8   37586661  Effectiveness of intravenous immunoglobulin th...
9   37698611  Virological evidence of the impact of non-phar...
10  37870070  Clinical diagnosis of SARS-CoV-2 infection: An...
11  37924841  Association of patient clinical and gut microb...
12  38030736  Effects of maternal type 1 diabetes and confou...
13  38180993  Serum lactate and mean arterial pressure thres...
14  38289867  Electrical storm treatment